In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import  math
from sklearn.model_selection import cross_validate,train_test_split
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import sys
from numpy import genfromtxt
from sklearn.cluster import DBSCAN

##### Read the data 

In [ ]:
encode = pd.read_csv("encode.csv")
clip = pd.read_csv("clip.csv") 
scenechange = pd.read_csv("scene_change.csv") 
scenechange.shape

#####  Add prefix to distinguish the tables after merging  

In [ ]:
clip=clip.add_prefix('clip_')

##### Calculate scene change percentage in one second

In [ ]:
scene_df = pd.DataFrame(columns=['clip_id','averagescenechange'])

In [ ]:
scenechange = scenechange[~scenechange['clip_id'].isnull()]
#scenechange.dropna(how='any',axis=0)
clip_id = scenechange['clip_id'].unique()
j = 0;
for i in clip_id:
    j = j + 1;
    scene_df.loc[j] = [i , np.sum(scenechange[scenechange['clip_id']==i]['percentage'])]


#####  Get the last part of the names in 'clip_filename' for simplification  

In [ ]:
clip= clip.dropna(subset=['clip_filename'])
size=clip['clip_filename'].shape[0]
for i in range(size):
    txt=clip['clip_filename'][i]
    out = txt.split("/")
    clip['clip_filename'][i]=out[-1]

##### Change the form of values from fractional to integer in 'clip_frame_rate' 

In [ ]:
size=clip['clip_frame_rate'].shape[0]
for i in range(size):
    txt=clip['clip_frame_rate'][i]
    out = txt.split("/")
    clip['clip_frame_rate'][i]=float(out[0])/float(out[1])
clip.head()

#####   Add prefix to distinguish the tables after merging   

In [ ]:
encode=encode.add_prefix('encode_')
#encode.head()

##### Merge the tables Clip and Encode

In [ ]:
encode_clip=encode.merge(clip, left_on='encode_clip_id', right_on='clip_id',how='left')
encode_clip.head()
encode_clip_scene = encode_clip.merge(scene_df,left_on='clip_id', right_on='clip_id',how='left')
#encode_clip.shape[1].

#####  Calculate the resolution, width times hight  

In [ ]:
encode_WidthHeight= encode_clip_scene['encode_width']*encode_clip_scene['encode_height']

##### As the clips duration are not the same, by dividing the 'averagescenechange' by 'clip_duration', the average scenechange for one second is calculated  

In [ ]:
scenechange_avg= encode_clip_scene['averagescenechange']/encode_clip_scene['clip_duration']

#####  Select the desired features from the merged tables  

In [ ]:
#df=pd.DataFrame({'encode_WidthHeight': encode_WidthHeight,'clip_filename':encode_clip['clip_filename'], 'encode_crf': encode_clip['encode_crf'], 'encode_bitrate_video': encode_clip['encode_bitrate_video'], 'encode_vmaf': encode_clip['encode_vmaf'], 'clip_duration': encode_clip['clip_duration'], 'clip_size': encode_clip['clip_size'], 'clip_bitrate_total': encode_clip['clip_bitrate_total'], 'clip_bitrate_video': encode_clip['clip_bitrate_video'], 'clip_height': encode_clip['clip_height'],'clip_frame_rate':encode_clip['clip_frame_rate']})
df=pd.DataFrame({'averagescenechange': scenechange_avg, 'encode_WidthHeight': encode_WidthHeight,'clip_filename':encode_clip_scene['clip_filename'], 'encode_crf': encode_clip_scene['encode_crf'], 'encode_bitrate_video': encode_clip_scene['encode_bitrate_video'], 'encode_vmaf': encode_clip_scene['encode_vmaf'], 'clip_duration': encode_clip_scene['clip_duration'], 'clip_size': encode_clip_scene['clip_size'], 'clip_height': encode_clip_scene['clip_height'],'clip_frame_rate':encode_clip_scene['clip_frame_rate']})
df.dropna(inplace=True)
df.head()

In [ ]:
df['clip_frame_rate'].describe()

##### Apply the method ITU-T P.1401 on 'encode_bitrate_video' to remove the extreme outliers

In [ ]:
shape=df['encode_bitrate_video'].shape[0]
IQR=df['encode_bitrate_video'].describe()[6]-df['encode_bitrate_video'].describe()[4]
IQR2=IQR*3
threshold=df['encode_bitrate_video'].describe()[6]+IQR2
for i in range(shape):
    if(df.iloc[i, 7:8].values[0] >= threshold):
        df.iloc[i, 7:8]=np.nan
        #df.at[i, 3:4] = 0

df.dropna(inplace=True)
df.head()

#####  Apply the method ITU-T P.1401 on 'averagescenechange' to remove the extreme outliers  

In [ ]:
shape=df['averagescenechange'].shape[0]
IQR=df['averagescenechange'].describe()[6]-df['averagescenechange'].describe()[4]
IQR2=IQR*3
threshold=df['averagescenechange'].describe()[6]+IQR2
for i in range(shape):
    if(df.iloc[i, 0:1].values[0] >= threshold):
        df.iloc[i, 0:1]=np.nan
        #df.at[i, 3:4] = 0

df.dropna(inplace=True)
df.shape

##### Apply the method ITU-T P.1401 on 'clip_size' to remove the extreme outliers 

In [ ]:
shape=df['clip_size'].shape[0]
IQR=df['clip_size'].describe()[6]-df['clip_size'].describe()[4]
IQR2=IQR*3
threshold=df['clip_size'].describe()[6]+IQR2
for i in range(shape):
    if(df.iloc[i, 5:6].values[0] >= threshold):
        df.iloc[i, 5:6]=np.nan
        #df.at[i, 3:4] = 0

df.dropna(inplace=True)
df.shape

In [ ]:
df.shape

#####  Apply the method ITU-T P.1401 on 'clip_duration' to remove the extreme outliers  

In [ ]:
shape=df['clip_duration'].shape[0]
IQR=df['clip_duration'].describe()[6]-df['clip_duration'].describe()[4]
IQR2=IQR*1.5
threshold=df['clip_duration'].describe()[6]+IQR2
for i in range(shape):
    if(df.iloc[i, 1:2].values[0] >= threshold):
        df.iloc[i, 1:2]=np.nan
        #df.at[i, 3:4] = 0

df.dropna(inplace=True)
df.head()

In [ ]:
df = df.drop(['clip_filename'], axis=1);

X = np.array(df.drop(['encode_vmaf'], 1))
X = preprocessing.scale(X)
y = np.array(df['encode_vmaf'])

##### Split the dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#####  Create SVR model  

SVR = svm.SVR()
SVR.fit(X_train, y_train)

In [ ]:
confidence = SVR.score(X_test, y_test)
print(confidence)

In [ ]:
predict_SVR=SVR.predict(X_test)
plt.scatter(y_test,predict_SVR)
plt.xlabel('Calculated vmaf')
plt.ylabel('Predicted vmaf')
plt.title('SVR Regression')
df_svr = pd.concat([pd.DataFrame(y_test) ,pd.DataFrame(predict_SVR)], axis = 1);
df_svr.to_excel('SVR_Regression.xlsx');

##### Calculate the correlation 

In [ ]:
print('Corr_SVR',np.corrcoef(y_test , predict_SVR)[0][1])

##### Calculate the MSE and RMSE 

In [ ]:
MSE_SVR = MSE(y_test, predict_SVR)
print('MSE: %f' % MSE_SVR)

rmse_SVR = sqrt(MSE_SVR)
print('RMSE: %f' % rmse_SVR)

##### Create Linear Regression model   

In [ ]:
LiReg = LinearRegression()
LiReg.fit(X_train, y_train)
confidence_LiReg = LiReg.score(X_test, y_test)
print(confidence_LiReg)

In [ ]:
predict_LiReg=LiReg.predict(X_test)

In [ ]:
plt.scatter(y_test,predict_LiReg)
plt.xlabel('Calculated vmaf')
plt.ylabel('Predicted vmaf')
plt.title('Linear Regression')

df_lin = pd.concat([pd.DataFrame(y_test) ,pd.DataFrame(predict_LiReg)], axis = 1);
df_lin.to_excel('Linear_Regression.xlsx');

#####  Calculate the correlation  

In [ ]:
print('Corr_Li',np.corrcoef(y_test , predict_LiReg)[0][1])

#####  Calculate the MSE and RMSE  

In [ ]:
MSE_Li=MSE(y_test, predict_LiReg)
print('MSE: %f' % MSE_Li)
rmse_Li = sqrt(MSE_Li)
print('RMSE: %f' % rmse_Li)

##### Create Random Forest model 

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train)
confidence_rf = rf.score(X_test, y_test)
print(confidence_rf)

In [ ]:
predict_rf=rf.predict(X_test)
plt.scatter(y_test,predict_rf)
plt.xlabel('Calculated vmaf')
plt.ylabel('Predicted vmaf')
plt.title('Random Forest Regression')
df_rf = pd.concat([pd.DataFrame(y_test) ,pd.DataFrame(predict_rf)], axis = 1);
df_rf.to_excel('RF_Regression.xlsx');

#####  Calculate the MSE and RMSE   

In [ ]:
print('Corr_rf',np.corrcoef(y_test , predict_rf)[0][1])
MSE_rf = MSE(y_test, predict_rf)
print('MSE: %f' % MSE_rf)
rmse_rf = sqrt(MSE_rf)
print('RMSE: %f' % rmse_rf)